In [21]:
from policyengine_us import Simulation
from policyengine_core.reforms import Reform
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from policyengine_core.charts import format_fig

In [22]:

reform = Reform.from_dict({
  "gov.aca.ptc_phase_out_rate[0].amount": {
    "2026-01-01.2100-12-31": 0
  },
  "gov.aca.ptc_phase_out_rate[1].amount": {
    "2025-01-01.2100-12-31": 0
  },
  "gov.aca.ptc_phase_out_rate[2].amount": {
    "2026-01-01.2100-12-31": 0
  },
  "gov.aca.ptc_phase_out_rate[3].amount": {
    "2026-01-01.2100-12-31": 0.02
  },
  "gov.aca.ptc_phase_out_rate[4].amount": {
    "2026-01-01.2100-12-31": 0.04
  },
  "gov.aca.ptc_phase_out_rate[5].amount": {
    "2026-01-01.2100-12-31": 0.06
  },
  "gov.aca.ptc_phase_out_rate[6].amount": {
    "2026-01-01.2100-12-31": 0.085
  },
  "gov.aca.ptc_income_eligibility[2].amount": {
    "2026-01-01.2100-12-31": True
  }
}, country_id="us")



In [23]:

situation_wv = {
  "people": {
    "you": {
      "age": {
        "2026": 64
      }
    },
    "your partner": {
      "age": {
        "2026": 64
      }
    },
    "your first dependent": {
      "age": {
        "2026": 17
      }
    }
  },
  "families": {
    "your family": {
      "members": [
        "you",
        "your partner",
        "your first dependent"
      ]
    }
  },
  "spm_units": {
    "your household": {
      "members": [
        "you",
        "your partner",
        "your first dependent"
      ]
    }
  },
  "tax_units": {
    "your tax unit": {
      "members": [
        "you",
        "your partner",
        "your first dependent"
      ]
    }
  },
  "households": {
   "your household": {
     "members": [
       "you",
       "your partner", 
       "your first dependent"  # All live in the same household
     ],
     "state_name": {
       "2026": "WV"  # Located in New York state
     },
    "county_fips": {
      "2026": "54005"
    }
   }
 },
  "marital_units": {
    "your marital unit": {
      "members": [
        "you",
        "your partner"
      ]
    },
    "your first dependent's marital unit": {
      "members": [
        "your first dependent"
      ],
      "marital_unit_id": {
        "2026": 1
      }
    }
  },
  "axes": [
    [
      {
        "name": "employment_income",
        "count": 800,
        "min": 0,
        "max": 600000
      }
    ]
  ]
}



In [24]:
simulation_wv = Simulation(
    situation=situation_wv,
)

reformed_simulation_wv = Simulation(
    situation=situation_wv,
    reform=reform,
)


In [25]:
import copy
import pandas as pd
from policyengine_us import Simulation

# ------------------------------------------------------------------
# 1.  Helper: run a one-point simulation and collect metrics
# ------------------------------------------------------------------
def get_metrics_for_income(base_situation: dict, income: float):
    """
    Returns baseline & reform PTC plus baseline Medicaid and CHIP metrics
    for a New York family of three at the specified annual income.

    Parameters
    ----------
    base_situation : dict
        Your `situation_wv` dictionary.
    income : float
        Total household employment income to test (USD, annual).

    Returns
    -------
    dict  with keys
        ptc_baseline       – ACA PTC in baseline
        ptc_ira_reform     – ACA PTC under IRA-style reform
        medicaid_cost      – household Medicaid benefit (baseline)
        per_capita_chip    – CHIP benefit ÷ household size (baseline)
    """
    # ---------------- Copy & inject the income --------------------
    sit = copy.deepcopy(base_situation)
    sit.pop("axes", None)                # single-point sim only

    # Split income equally between both adults
    for person in ["you", "your partner"]:
        sit["people"][person]["employment_income"] = {"2026": income / 2}

    hh_size = len(sit["people"])

    # ---------------- Run simulations ----------------------------
    sim_base   = Simulation(situation=sit)
    sim_reform = Simulation(situation=sit, reform=reform)

    # ---------------- Pull variables -----------------------------
    # ACA PTC
    ptc_base   = sim_base.calculate("aca_ptc",   map_to="household", period=2026)[0]
    ptc_reform = sim_reform.calculate("aca_ptc", map_to="household", period=2026)[0]
    SLCSP = sim_base.calculate("slcsp", map_to="household", period=2026)[0]

    # Medicaid benefit (adult or child)
    medicaid_cost = sim_base.calculate("medicaid_cost", map_to="household", period=2026)[0]

    # CHIP benefit – variable names differ by PE-US version:
    #   * If your build has `chip_cost`, use that.
    #   * Otherwise use `chip` (total CHIP dollars) or adjust as needed.
    chip_total = sim_base.calculate("per_capita_chip", map_to="household", period=2026)[0]
    per_capita_chip = chip_total / hh_size if hh_size else 0

    return dict(
        ptc_baseline     = ptc_base,
        ptc_ira_reform   = ptc_reform,
        medicaid_cost    = medicaid_cost,
        per_capita_chip  = per_capita_chip,
        SLCSP = SLCSP
    )

# ------------------------------------------------------------------
# 2.  Income targets (family of 3, 2026 FPL thresholds you supplied)
# ------------------------------------------------------------------
targets_wv = {
    "154 % FPL ($41,041)" : 41_041,
    "200 % FPL ($53,300)" : 53_300,
    "300 % FPL ($79,950)": 79_950,
    "405 % FPL ($107,933)": 107_933,
}

rows = []
for label, inc in targets_wv.items():
    metrics = get_metrics_for_income(situation_wv, inc)
    rows.append(
        dict(income_label = label, income_usd = inc, **metrics)
    )

wv_ptc_df = pd.DataFrame(rows)
wv_ptc_df


,income_label,income_usd,ptc_baseline,ptc_ira_reform,medicaid_cost,per_capita_chip,SLCSP
0,"154 % FPL ($41,041)",41041,51890.406250,53541.898438,0.0,767.5,53607.5625
1,"200 % FPL ($53,300)",53300,50249.664062,52541.562500,0.0,767.5,53607.5625
2,"300 % FPL ($79,950)",79950,46012.312500,48810.562500,0.0,767.5,53607.5625
3,"405 % FPL ($107,933)",107933,0.000000,52340.558594,0.0,0.0,0.0000


In [26]:
household_income_wv = simulation_wv.calculate("employment_income", map_to="household", period=2026)
baseline_wv_per_capita_chip = simulation_wv.calculate("per_capita_chip", map_to="household", period=2026)
baseline_wv_aca_ptc = simulation_wv.calculate("aca_ptc", map_to="household", period=2026)
baseline_wv_medicaid_cost = simulation_wv.calculate("medicaid_cost", map_to="household", period=2026)
baseline_wv_net_income_including_health_benefits = simulation_wv.calculate("household_net_income_including_health_benefits", map_to="household", period=2026)
baseline_wv_slcsp = simulation_wv.calculate("slcsp", map_to="household", period=2026)

reform_wv_per_capita_chip = reformed_simulation_wv.calculate("per_capita_chip", map_to="household", period=2026)
reform_wv_aca_ptc = reformed_simulation_wv.calculate("aca_ptc", map_to="household", period=2026)
reform_wv_medicaid_cost = reformed_simulation_wv.calculate("medicaid_cost", map_to="household", period=2026)
reform_wv_net_income_including_health_benefits = reformed_simulation_wv.calculate("household_net_income_including_health_benefits", map_to="household", period=2026)







# Calculate total benefits for each scenario
baseline_wv_total = [sum(x) for x in zip(baseline_wv_per_capita_chip, baseline_wv_aca_ptc, baseline_wv_medicaid_cost)]
reform_wv_total = [sum(x) for x in zip(reform_wv_per_capita_chip, reform_wv_aca_ptc, reform_wv_medicaid_cost)]




In [27]:
GRAY = "#808080"
BLUE_PRIMARY = "#2C6496"
TEAL_ACCENT = "#39C6C0"
DARK_GRAY = "#616161"

In [28]:
# Create wv graph
fig_wv = go.Figure()

# Add baseline traces (solid lines)
fig_wv.add_trace(go.Scatter(
    x=household_income_wv, 
    y=baseline_wv_per_capita_chip, 
    mode='lines', 
    name='CHIP (Baseline)', 
    line=dict(color=GRAY, width=2)
))

fig_wv.add_trace(go.Scatter(
    x=household_income_wv, 
    y=baseline_wv_aca_ptc, 
    mode='lines', 
    name='ACA PTC (Baseline)', 
    line=dict(color=BLUE_PRIMARY, width=2)
))

fig_wv.add_trace(go.Scatter(
    x=household_income_wv, 
    y=baseline_wv_medicaid_cost, 
    mode='lines', 
    name='Medicaid (Baseline)', 
    line=dict(color=TEAL_ACCENT, width=2)
))

# Add reform traces (dotted lines)
fig_wv.add_trace(go.Scatter(
    x=household_income_wv, 
    y=reform_wv_per_capita_chip, 
    mode='lines', 
    name='CHIP (Reform)', 
    line=dict(color=GRAY, width=2, dash='dot')
))

fig_wv.add_trace(go.Scatter(
    x=household_income_wv, 
    y=reform_wv_aca_ptc, 
    mode='lines', 
    name='ACA PTC (Reform)', 
    line=dict(color=BLUE_PRIMARY, width=2, dash='dot')
))

fig_wv.add_trace(go.Scatter(
    x=household_income_wv, 
    y=reform_wv_medicaid_cost, 
    mode='lines', 
    name='Medicaid (Reform)', 
    line=dict(color=TEAL_ACCENT, width=2, dash='dot')
))

# Add total lines
fig_wv.add_trace(go.Scatter(
    x=household_income_wv, 
    y=baseline_wv_total, 
    mode='lines', 
    name='Total Benefits (Baseline)', 
    line=dict(color=DARK_GRAY, width=2)
))

fig_wv.add_trace(go.Scatter(
    x=household_income_wv, 
    y=reform_wv_total, 
    mode='lines', 
    name='Total Benefits (Reform)', 
    line=dict(color=DARK_GRAY, width=2, dash='dot')
))

# Update layout
fig_wv.update_layout(
    title='WV Household (Family of 3) - Program Benefits by Income Level',
    xaxis_title='Household Income',
    yaxis_title='Benefit Amount',
    legend_title='Programs',
    xaxis=dict(tickformat='$,.0f', range=[0, 600000]),
    yaxis=dict(tickformat='$,.0f'),
    height=600,
    width=1000
)



In [29]:
#House hold net income graphs
import plotly.graph_objects as go

# ---------- wv fam  ----------
fig_wv = go.Figure()

# Baseline (solid)
fig_wv.add_trace(go.Scatter(
    x=household_income_wv,
    y=baseline_wv_net_income_including_health_benefits,
    mode='lines',
    name='Health Net Income (Baseline)',
    line=dict(color=DARK_GRAY, width=2)        # use your palette constant
))

# Reform (dotted)
fig_wv.add_trace(go.Scatter(
    x=household_income_wv,
    y=reform_wv_net_income_including_health_benefits,
    mode='lines',
    name='Health Net Income (Reform)',
    line=dict(color=DARK_GRAY, width=2, dash='dot')
))

# Layout
fig_wv.update_layout(
    title='New York Household (Family of 3) – Health-Adjusted Net Income by Household Income',
    xaxis_title='Household Income',
    yaxis_title='Health-Adjusted Net Income',
    legend_title='Scenario',
    xaxis=dict(tickformat='$,.0f', range=[0, 400_000]),
    yaxis=dict(tickformat='$,.0f'),
    height=600,
    width=1000
)

# Optional wrapper if you use one elsewhere
fig_wv = format_fig(fig_wv)

fig_wv.show()

# --- Δ Health-adjusted net income (Reform – Baseline) ---
delta_wv = (
    reform_wv_net_income_including_health_benefits
    - baseline_wv_net_income_including_health_benefits
)

fig_delta_wv = go.Figure()

fig_delta_wv.add_trace(go.Scatter(
    x=household_income_wv,
    y=delta_wv,
    mode='lines',
    name='Δ Net Income (Reform – Baseline)',
    line=dict(color=DARK_GRAY, width=2)
))

fig_delta_wv.update_layout(
    title='New York Household (Family of 3) – Impact of Extending Enhanced Premium Tax Credits',
    xaxis_title='Household Income',
    yaxis_title='Δ Net Income',
    xaxis=dict(tickformat='$,.0f', range=[0, 400_000]),
    yaxis=dict(tickformat='$,.0f', zeroline=True, zerolinewidth=1),
    height=600,
    width=1000,
    legend=dict(orientation='h')
)

fig_delta_wv = format_fig(fig_delta_wv)  # if you use the helper elsewhere
fig_delta_wv.show()


In [30]:
# ---------- texas couple ----------
fig_tx = go.Figure()

# Baseline (solid)
fig_tx.add_trace(go.Scatter(
    x=household_income_texas,
    y=baseline_texas_net_income_including_health_benefits,
    mode='lines',
    name='Health Net Income (Baseline)',
    line=dict(color=DARK_GRAY, width=2)        # use your palette constant
))

# Reform (dotted)
fig_tx.add_trace(go.Scatter(
    x=household_income_texas,
    y=reform_texas_net_income_including_health_benefits,
    mode='lines',
    name='Health Net Income (Reform)',
    line=dict(color=DARK_GRAY, width=2, dash='dot')
))

# Layout
fig_tx.update_layout(
    title='Texas Household (Married Couple) – Health-Adjusted Net Income by Household Income',
    xaxis_title='Household Income',
    yaxis_title='Health-Adjusted Net Income',
    legend_title='Scenario',
    xaxis=dict(tickformat='$,.0f', range=[0, 200_000]),
    yaxis=dict(tickformat='$,.0f'),
    height=600,
    width=1000
)

# Optional wrapper if you use one elsewhere
fig_tx = format_fig(fig_tx)

fig_tx.show()
# --- Δ Health-adjusted net income (Reform – Baseline), Texas ---
delta_tx = (
    reform_texas_net_income_including_health_benefits
    - baseline_texas_net_income_including_health_benefits
)

fig_delta_tx = go.Figure()

fig_delta_tx.add_trace(go.Scatter(
    x=household_income_texas,
    y=delta_tx,
    mode='lines',
    name='Δ Net Income (Reform – Baseline)',
    line=dict(color=DARK_GRAY, width=2)
))

fig_delta_tx.update_layout(
    title='Texas Household (Family of 3) – Impact of Extending Enhanced Premium Tax Credits',
    xaxis_title='Household Income',
    yaxis_title='Δ Net Income',
    xaxis=dict(tickformat='$,.0f', range=[0, 200_000]),
    yaxis=dict(tickformat='$,.0f', zeroline=True, zerolinewidth=1),
    height=600,
    width=1000,
    legend=dict(orientation='h')
)

fig_delta_tx = format_fig(fig_delta_tx)  # if you’re using that helper
fig_delta_tx.show()



NameError: name 'household_income_texas' is not defined

In [14]:
# ---------- Pull the inputs ----------
household_income_wv = simulation_wv.calculate(
    "employment_income", map_to="household", period=2026
)

baseline_raw = simulation_wv.calculate(
    "marginal_tax_rate_including_health_benefits",
    map_to="household",
    period=2026
)

reform_raw = reformed_simulation_wv.calculate(
    "marginal_tax_rate_including_health_benefits",
    map_to="household",
    period=2026
)

# ---------- Limit MRT values to ±100 % ----------
baseline_mtr = np.clip(baseline_raw, -1, 1)   # -1 ↔ –100 %, 1 ↔ 100 %
reform_mtr   = np.clip(reform_raw,   -1, 1)

# ---------- Build the graph ----------
fig_wv_mtr = go.Figure()

fig_wv_mtr.add_trace(go.Scatter(
    x=household_income_wv,
    y=baseline_mtr,
    mode='lines',
    name='Marginal Tax Rate (Baseline)',
    line=dict(color=DARK_GRAY, width=2)
))

fig_wv_mtr.add_trace(go.Scatter(
    x=household_income_wv,
    y=reform_mtr,
    mode='lines',
    name='Marginal Tax Rate (Reform)',
    line=dict(color=BLUE_PRIMARY, width=2, dash='dot')
))

fig_wv_mtr.update_layout(
    title='New York Household (Family of 3) – Marginal Tax Rate Including Health Benefits by Household Income',
    xaxis_title='Household Income',
    yaxis_title='Marginal Tax Rate (Including Health Benefits)',
    legend_title='Scenario',
    xaxis=dict(tickformat='$,.0f', range=[0, 200_000]),
    yaxis=dict(tickformat='.0%', range=[-1, 1]),  # keep the same visual bounds
    height=600,
    width=1000
)

fig_wv_mtr = format_fig(fig_wv_mtr)
fig_wv_mtr.show()


In [15]:
import numpy as np

# ---------- Pull the inputs ----------
household_income_texas = simulation_texas.calculate(
    "employment_income", map_to="household", period=2026
)

baseline_raw = simulation_texas.calculate(
    "marginal_tax_rate_including_health_benefits",
    map_to="household",
    period=2026
)

reform_raw = reformed_simulation_texas.calculate(
    "marginal_tax_rate_including_health_benefits",
    map_to="household",
    period=2026
)

# ---------- Limit MRT values to ±100 % ----------
baseline_texas_mtr_including_health_benefits = np.clip(baseline_raw, -1, 1)
reform_texas_mtr_including_health_benefits   = np.clip(reform_raw,   -1, 1)

# ---------- Build the graph ----------
fig_texas_mtr = go.Figure()

fig_texas_mtr.add_trace(go.Scatter(
    x=household_income_texas,
    y=baseline_texas_mtr_including_health_benefits,
    mode='lines',
    name='Marginal Tax Rate (Baseline)',
    line=dict(color=DARK_GRAY, width=2)
))

fig_texas_mtr.add_trace(go.Scatter(
    x=household_income_texas,
    y=reform_texas_mtr_including_health_benefits,
    mode='lines',
    name='Marginal Tax Rate (Reform)',
    line=dict(color=BLUE_PRIMARY, width=2, dash='dot')
))

fig_texas_mtr.update_layout(
    title='Texas Household (Couple) – Marginal Tax Rate Including Health Benefits by Household Income',
    xaxis_title='Household Income',
    yaxis_title='Marginal Tax Rate (Including Health Benefits)',
    legend_title='Scenario',
    xaxis=dict(tickformat='$,.0f', range=[0, 200_000]),
    yaxis=dict(tickformat='.0%', range=[-1, 1]),  # stays consistent with the clipping
    height=600,
    width=1000
)

fig_texas_mtr = format_fig(fig_texas_mtr)
fig_texas_mtr.show()
